In [176]:
# Importing essential modules
import pandas as pd
import sqlite3
from plotly import express as px
from sklearn.linear_model import LinearRegression
from plotly.io import write_html

# Creating a database

In [50]:
# Read in the temperature df
temps = pd.read_csv("temps_stacked.csv")
# Read in the countries df
countries = pd.read_csv('countries.csv')
# Read in the stations df directly from url
url = "https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/station-metadata.csv"
stations = pd.read_csv(url)

# Substitute white space for underscore in df column names
countries = countries.rename(columns= {"FIPS 10-4": "FIPS_10_4"})
countries = countries.rename(columns= {"ISO 3166": "ISO_3166"})

In [51]:
# Create and connect to a temps.db database
conn = sqlite3.connect("temps.db")

# Add the three tables to temps.db
temps.to_sql("temperatures", conn, if_exists="replace", index=False)
stations.to_sql("stations", conn, if_exists="replace", index=False)
countries.to_sql("countries", conn, if_exists="replace", index=False)

# close your connection after database construction
conn.close()

In [36]:
temps.head(5)

,ID,Year,Month,Temp
0,ACW00011604,1961,1,-0.89
1,ACW00011604,1961,2,2.36
2,ACW00011604,1961,3,4.72
3,ACW00011604,1961,4,7.73
4,ACW00011604,1961,5,11.28


In [193]:
countries.tail(5)

,FIPS_10_4,ISO_3166,Name
274,-,-,World
275,YM,YE,Yemen
276,-,-,Zaire
277,ZA,ZM,Zambia
278,ZI,ZW,Zimbabwe


In [38]:
stations.head(5)

,ID,LATITUDE,LONGITUDE,STNELEV,NAME
0,ACW00011604,57.7667,11.8667,18.0,SAVE
1,AE000041196,25.3330,55.5170,34.0,SHARJAH_INTER_AIRP
2,AEM00041184,25.6170,55.9330,31.0,RAS_AL_KHAIMAH_INTE
3,AEM00041194,25.2550,55.3640,10.4,DUBAI_INTL
4,AEM00041216,24.4300,54.4700,3.0,ABU_DHABI_BATEEN_AIR


# Query Function

In [90]:
def query_climate_database(country, year_begin, year_end, month):
    # Connect to the database
    conn = sqlite3.connect("temps.db")
    # SQL command for joining the tables and returning the desired columns
    cmd = '''
    SELECT
        s.NAME, s.LATITUDE, s.LONGITUDE, c.Name Country, t.Year, t.Month, t.Temp
    FROM
        temperatures t
    LEFT JOIN stations s ON t.ID = s.ID
    LEFT JOIN countries c ON SUBSTR(t.ID, 1, 2) = c.FIPS_10_4
    '''
    # Execute the SQL command and store the queried data into df
    df = pd.read_sql_query(cmd, conn)
    conn.close()
    # Return entries that follow user specifications
    return df.loc[(df.Country == country) & (df.Month == month) &
                  (df.Year >= year_begin) & (df.Year <= year_end)].reset_index().drop('index', axis = 1)

In [91]:
df = query_climate_database(country = "India", 
                       year_begin = 1980, 
                       year_end = 2020,
                       month = 1)

In [103]:
df

,NAME,LATITUDE,LONGITUDE,Country,Year,Month,Temp
0,PBO_ANANTAPUR,14.583,77.633,India,1980,1,23.48
1,PBO_ANANTAPUR,14.583,77.633,India,1981,1,24.57
2,PBO_ANANTAPUR,14.583,77.633,India,1982,1,24.19
3,PBO_ANANTAPUR,14.583,77.633,India,1983,1,23.51
4,PBO_ANANTAPUR,14.583,77.633,India,1984,1,24.81
...,...,...,...,...,...,...,...
3147,DARJEELING,27.050,88.270,India,1983,1,5.10
3148,DARJEELING,27.050,88.270,India,1986,1,6.90
3149,DARJEELING,27.050,88.270,India,1994,1,8.10
3150,DARJEELING,27.050,88.270,India,1995,1,5.60


# Geographic Scatter Function for Yearly Temperature Increases

In [131]:
# Function that takes in a data frame (with Year and Temp columns)
# Regresses on the data points and returns the best fit line's slope
def coef(data_group):
    X = data_group[["Year"]] # expect data frame, not series
    y = data_group["Temp"]
    LR = LinearRegression()
    LR.fit(X, y)
    slope = LR.coef_[0]
    return slope

In [187]:
def temperature_coefficient_plot(country, year_begin, year_end, month, min_obs, **kwargs):
    raw = query_climate_database(country = country, 
                                 year_begin = year_begin, 
                                 year_end = year_end,
                                 month = month)
    freq = pd.DataFrame(raw.groupby(['NAME', 'Month'])['Temp'].transform(len)).rename(columns={'Temp':'freq'})
    raw = raw.join(freq)
    filtered = raw[raw.freq >= min_obs]
    temp_change = filtered.groupby(["NAME", "Month"]).apply(coef).reset_index()
    temp_change = temp_change.rename(columns = {0:"slope"}).round(decimals = 4)
    df = temp_change.merge(filtered[["NAME", "LATITUDE", "LONGITUDE"]], left_on = "NAME", right_on = "NAME")
    fig = px.scatter_mapbox(df, # data for the points you want to plot
                            lat = "LATITUDE", # column name for latitude informataion
                            lon = "LONGITUDE", # column name for longitude information
                            hover_name = "NAME", # what's the bold text that appears when you hover over
                            color="slope", # represent temp using color
                            labels = {"slope":"Estimated Yearly Increase"},
                            **kwargs) # remaining user-specified arguments
    fig.update_layout(margin={"r":0,"t":50,"l":20,"b":0})
    return fig

In [188]:
# assumes you have imported necessary packages
color_map = px.colors.diverging.RdGy_r # choose a colormap

fig1 = temperature_coefficient_plot("India", 1980, 2020, 1, min_obs = 10,
                                   zoom = 2,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map,
                                   title = "Estimates of yearly change in Temperature in January"\
                                   "for stations in India, years 1980-2020",
                                   color_continuous_midpoint = 0)

fig1.show()

In [189]:
# assumes you have imported necessary packages
color_map = px.colors.diverging.RdGy_r # choose a colormap

fig2 = temperature_coefficient_plot("China", 1990, 2020, 1, min_obs = 5,
                                   zoom = 3,
                                   mapbox_style="carto-darkmatter",
                                   color_continuous_scale=color_map,
                                   title = "Estimates of yearly change in Temperature in January"\
                                   "for stations in China, years 1990-2020",
                                   color_continuous_midpoint = 0)

fig2.show()

In [190]:
write_html(fig1, "geographic_scatter_india.html")
write_html(fig2, "geographic_scatter_china.html")

In [ ]:
conn = sqlite3.connect("temps.db")
cmd = '''
    SELECT
        c.Name Country, s.NAME station, t.Year, t.Temp, s.STNELEV
    FROM
        temperatures t
    LEFT JOIN stations s ON t.ID = s.ID
    LEFT JOIN countries c ON SUBSTR(t.ID, 1, 2) = c.FIPS_10_4
    '''
    # Execute the SQL command and store the queried data into df
df = pd.read_sql_query(cmd, conn)
conn.close()
df

# Another query function

In [214]:
def query_climate_database2(countries, year_begin, year_end):
    # Connect to the database
    conn = sqlite3.connect("temps.db")
    # SQL command for joining the tables and returning the desired columns
    cmd = '''
    SELECT
        c.Name Country, s.NAME station, t.Year, t.Month, t.Temp, s.STNELEV
    FROM
        temperatures t
    LEFT JOIN stations s ON t.ID = s.ID
    LEFT JOIN countries c ON SUBSTR(t.ID, 1, 2) = c.FIPS_10_4
    '''
    # Execute the SQL command and store the queried data into df
    df = pd.read_sql_query(cmd, conn)
    conn.close()
    # Return entries that follow user specifications
    return df.loc[(df.Country.isin(countries))& (df.Year >= year_begin) & (df.Year <= year_end)].reset_index().drop('index', axis = 1)

In [217]:
df2 = query_climate_database2(["China", "United States", "India"], 2010, 2020)

In [218]:
df2.tail(10)

,Country,station,Year,Month,Temp,STNELEV
945465,United States,LINCOLN_11_SW,2020,3,6.47,418.2
945466,United States,LINCOLN_11_SW,2020,4,9.73,418.2
945467,United States,LINCOLN_11_SW,2020,5,14.81,418.2
945468,United States,LINCOLN_11_SW,2020,6,24.13,418.2
945469,United States,LINCOLN_11_SW,2020,7,24.44,418.2
945470,United States,LINCOLN_11_SW,2020,8,23.29,418.2
945471,United States,LINCOLN_11_SW,2020,9,17.97,418.2
945472,United States,LINCOLN_11_SW,2020,10,9.48,418.2
945473,United States,LINCOLN_11_SW,2020,11,7.23,418.2
945474,United States,LINCOLN_11_SW,2020,12,-0.51,418.2


# Multi-facet

In [213]:
pd.DataFrame(df2.groupby(["Country", "Year"])["Temp"].mean()).reset_index()

,Country,Year,Temp
0,China,1980,11.024547
1,China,1981,11.068464
2,China,1982,11.382063
3,China,1983,11.333335
4,China,1984,10.774125
...,...,...,...
118,United States,2016,11.848855
119,United States,2017,11.605787
120,United States,2018,11.094212
121,United States,2019,10.668990


In [219]:
fig = px.scatter(data_frame = df2, # data that needs to be plotted
                 x = "Month", # column name for x-axis
                 y = "Temp", # column name for y-axis
                 color = "Country", # column name for color coding
                 opacity = 0.5,
                 width = 750,
                 height = 600,
                facet_col = "STNELEV",
                facet_row = "Year") # column name

# reduce whitespace
fig.update_layout(margin={"r":0,"t":20,"l":0,"b":0})
# show the plot
fig.show()

ValueError: Horizontal spacing cannot be greater than (1 / (cols - 1)) = 0.000286.
The resulting plot would have 3499 columns (cols=3499).
Use the facet_col_spacing argument to adjust this spacing.

# Alluvial diagram

In [ ]:
colors = {"China"    : "#2a9d8f",
          "United States" : "#e9c46a",
          "India"    : "#e76f51"}

color_hex = df2["Country"].map(colors)

###

fig = px.parallel_categories(df2,
                             dimensions = ["Species", "Island", "Sex"], # order matters here
                             color = color_hex,
                             height = 300)


fig.update_layout(margin={"r":20,"t":0,"l":20,"b":0})
fig.show()